In [1]:
import os
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
mode = 'read' # 'write' or 'read'
if mode == 'write':
    # To write the data
    path_to_data = 'data/'
    file_names = os.listdir(path_to_data)
    print('Number of files: ', len(file_names))
    data = []
    for i in tqdm(range(len(file_names))):
        with open(path_to_data + file_names[i], 'r') as f:
            data.append(BeautifulSoup(json.loads(f.read())['data'], 'html.parser').get_text())
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f)
else:
    # To read the data
    with open('data.pkl', 'rb') as f:
        data = pickle.load(f)

In [3]:
data[0]

' T.C.İSTANBUL5. ASLİYE TİCARET MAHKEMESİ ESAS NO\t: 2016/451 EsasKARAR NO\t: 2019/85DAVA\t: Alacak (Ticari Nitelikteki Hizmet Sözleşmesinden Kaynaklanan)DAVA TARİHİ\t: 21/04/2016KARAR TARİHİ\t: 05/02/2019Mahkememizde görülmekte olan Alacak davasının yapılan açık yargılaması sonunda,GEREĞİ DÜŞÜNÜLDÜ:Davacı vekili 21/04/2016 tarihli dava dilekçesinde; Taraflar ve tarafların dışındaki kişilerle yapılmış 20 Mart 2006 tarihli düzenleme şeklinde ortaklar özel sözleşmesi gereğince davalı ve dava dışı diğer şahısların taşınmazlarının kurulacak bir şirkete devredilmesi ve kurulmuş şirketin bu taşınmazlar üzerinde inşaat faaliyetine başlamasını öngören bir sözleşme yapıldığını, bu sözleşmeye göre aralarında davalının da bulunduğu "mal sahiplerine" ait taşınmazların üzerinde yapılacak inşaatın finansmanının davacı tarafından karşılanacağını, taşınmaz üzerindeki borçların ve bazı giderlerin davacı tarafından ödendiğini, yapılacak inşaata ilişkin, öngörülen biçimde ruhsat alınmasının yasal olarak 